In [24]:
import os
import time
import numpy as np
import soundfile as sf
import simpleaudio as sa
from datetime import datetime
import av

# Path to the folder with recordings
RECORDINGS_DIR = "./recordings"


In [38]:
def play_opus(file_path):
    try:
        # Open Opus file with PyAV (FFmpeg backend)
        container = av.open(file_path)
        stream = next(s for s in container.streams if s.type == 'audio')

        # Decode audio frames
        audio_frames = []
        sample_rate = stream.sample_rate
        num_channels = stream.channels or 1  # Default to mono if not specified

        for frame in container.decode(stream):
            frame_data = frame.to_ndarray()

            # Convert stereo to mono if necessary
            if frame_data.ndim > 1:
                frame_data = np.mean(frame_data, axis=0)  # Convert to mono by averaging channels

            # Convert float32 data to int16 PCM (sound card expects this)
            if frame_data.dtype == np.float32:
                frame_data = (frame_data * 32767).astype(np.int16)

            audio_frames.append(frame_data)

        # Concatenate all frames into a single NumPy array
        if len(audio_frames) == 0:
            print(f"❌ Error: No valid audio frames in {file_path}")
            return

        audio_data = np.concatenate(audio_frames).astype(np.int16)

        # Play audio using simpleaudio
        play_obj = sa.play_buffer(audio_data, num_channels=num_channels, bytes_per_sample=2, sample_rate=sample_rate)
        play_obj.wait_done()  # Wait for playback to finish

    except Exception as e:
        print(f"❌ Error playing {file_path}: {e}")

In [41]:
play_opus("recordings/minut.opus")

In [42]:
# POUHE PREHRAVANI AUDIO SOUBORU V SEKVENCI
play_opus("recordings/je_prave.opus")
play_opus("recordings/20.opus")
play_opus("recordings/2.opus")
play_opus("recordings/hodin.opus")
play_opus("recordings/30.opus")
play_opus("recordings/4.opus")
play_opus("recordings/minut.opus")

In [ ]:
# TODO: TRIM AND MERGE THE BINARY DATA INTO 1 SEQUENCE. ALSO NORMALIZE THE VOLUME
# normalization would be done through some linear regression or something like that
# also normalize pitch and tempo
# add an option to switch to a female voice